In [1]:
import xml.etree.ElementTree as ET
import numpy
import pandas
import nltk
import csv
import numpy
import sklearn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Step 1: Data Pre-Processing
<div style="direction:rtl  ; font-family: 'Nunito', sans-serif;" > 
    
برای اجرای کدها بایستی که فایل‌های train_data.xml، dev_data.xml و test_data.xml که همان فایل‌های اولیه ارائه شده در تمرین است و نیز فایل‌های pa_pp_train_data.csv، pa_pp_dev_data.csv و pa_pp_test_data.csv در کنار فایل نوت بوک ژوپیتر قرار داشته باشند.
    
    
ابتدا مراحل پیش پردازش را به این صورت انجام می‌دهیم که نخست متن سوالات و پاسخ‌های ورودی را با کمک کتابخانه nltk توکن‌توکن می‌کنیم. بعد از حذف استاپ ورد‌ها، کوچک‌سازی حروف و استم کردن با PorterStemmer را بازهم با استفاده از کتابخانه nltk انجام می‌دهیم در ادامه بخش‌های مورد نیاز از داده‌ها یعنی:
•	RELQ_ID
•	RelQBody
•	RELC_ID
•	RELC_RELEVANCE2RELQ
•	RelCText
را از فایل‌های train_data.xml، dev_data.xml و test_data.xml جدا می‌کنیم. برای صفت RelCText به ازای مقدار Good عدد 2، PotentiallyUseful عدد 1 و به ازای Bad عدد 0 در نظر می‌گیریم و به صورت مناسب به ترتیب در فایل‌های pa_pp_train_data.csv، pa_pp_dev_data.csv و pa_pp_test_data.csv ذخیره می‌کنیم.

</div>

In [2]:
ANSWER_LABELS_MAPPING = {'False': 0, 'True': 1, 'NonFactual': 2}
label_array = ['Bad', 'PotentiallyUseful','Good']
test_label_array = ['?']
QUESTION_LABELS_MAPPING = {'Opinion': 0, 'Factual': 1, 'Socializing': 2}

In [3]:
def get_label(original_label, label_mapping):
    if original_label in label_mapping.keys():
        return label_mapping[original_label]
    return -1

In [4]:
def read_answer_labels_from_xml(input_xml_file):
    results = []
    print('parsing...', input_xml_file)

    tree = ET.parse(input_xml_file)
    root = tree.getroot()
    for thread in root:
        question_tag = thread[0]
        # question_fact_label = question_tag.attrib['RELQ_FACT_LABEL']
        # if question_fact_label == 'Factual':
        for index, answer_tag in enumerate(thread):
            if index ==0 :
                question_body = answer_tag[1].text
                question_number = answer_tag.attrib['RELQ_ID']
                if(question_body==None):
                    question_body="none"
            if index > 0: # the 0 index was processed above - it is the question
                # answer_fact_label = answer_tag.attrib['RELC_FACT_LABEL']
                answer_id = answer_tag.attrib['RELC_ID']
                answer_body = answer_tag[0].text
                answer_number = answer_tag.attrib['RELC_ID']
                if(answer_body==None):
                    answer_body="none"
                # label = get_label(answer_fact_label, ANSWER_LABELS_MAPPING)
                label = test_label_array.index(answer_tag.attrib['RELC_RELEVANCE2RELQ'])

                # if label > -1:
                results.append([question_number,question_body,answer_number,answer_body,label])
    return results

In [5]:
# Reads answer labels from file in the task input format.
def read_question_labels_from_xml(input_xml_file):
    labels = []
    print('parsing...', input_xml_file)
    tree = ET.parse(input_xml_file)
    root = tree.getroot()
    for thread in root:
        question_tag = thread[0]
        question_id = question_tag.attrib['RELQ_ID']
        question_fact_label = question_tag.attrib['RELQ_FACT_LABEL']
        question_body = thread[0][1].text
        label = get_label(question_fact_label, QUESTION_LABELS_MAPPING)
        item = []
        item.append(question_id)
        item.append(question_body)
        item.append(label)
        labels.append(item)
    return labels

In [6]:
def get_no_id_labels(labels):
    no_id_labels = map(lambda item:list(filter(lambda x:item.index(x)!=0,item)),labels)
    return list(no_id_labels)

In [7]:
result = read_answer_labels_from_xml("train_data.xml")

parsing... train_data.xml


ValueError: 'Good' is not in list

In [ ]:
dev_result = read_answer_labels_from_xml("test_data.xml")

In [ ]:
stop_words = set(stopwords.words('english'))

dicts = []
corpus = []
labels = []

output = [["", "", "", "", ""]]

for line in dev_result:
    tokens = nltk.word_tokenize(line[1])
    filtered_sentence = [w for w in tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    words = [word for word in filtered_sentence if word.isalpha()]
    porter = PorterStemmer()
    q_stemmed = [porter.stem(word) for word in words]

    tokens = nltk.word_tokenize(line[3])
    filtered_sentence = [w for w in tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    words = [word for word in filtered_sentence if word.isalpha()]
    porter = PorterStemmer()
    a_stemmed = [porter.stem(word) for word in words]
    

    dict = {
        "question number": line[0],
        "question": q_stemmed,
        "answer number": line[2],
        "answer": a_stemmed,
        "Label": line[4],
    }
    dicts.append(dict)
    
    q_string_tokens=' '.join([str(elem) for elem in q_stemmed])
    a_string_tokens=' '.join([str(elem) for elem in a_stemmed])

    temp = [line[0], q_string_tokens, line[2], a_string_tokens, line[4]]
    output.append(temp)

import json
with open('outputfile.csv', 'w') as fout:
    json.dump(dicts, fout)

In [78]:
'''
pandas_data = pandas.DataFrame(output)
pandas_data.to_csv("pa_pp_train_data.csv",sep=",",header=True,index=False,encoding="utf-8")
'''

In [13]:
'''
pandas_data = pandas.DataFrame(output)
pandas_data.to_csv("pa_pp_test_data.csv",sep=",",header=True,index=False,encoding="utf-8")
'''

In [8]:
def get_KB_selector(func, X, y, k):
    if func == "chi2":
        return SelectKBest(chi2, k=k).fit(X, y)
    if func == "f_classif":
        return SelectKBest(f_classif, k=k).fit(X, y)
    if func == "mutual_info_classif":
        return SelectKBest(mutual_info_classif, k=k).fit(X, y)

In [9]:
'''
||
>>>>>>>
'''
# pp_train_data
pp_train_data_file = open('pa_pp_train_data.csv', 'r', encoding = "utf8")
pp_train_data_reader = csv.reader(pp_train_data_file)
binary_label = []
for line in pp_train_data_reader:
    if line[4] == '2':
        binary_label.append(1)
    else:
        binary_label.append(0)
# print (binary_label)
 
pp_train_data_file_2 = open('pa_pp_train_data.csv', 'r', encoding = "utf8")
pp_train_data_reader_2 = csv.reader(pp_train_data_file_2)

questions_number_array = []
questions_array = []
answer_number_array = []
answer_array = []
label_array = []

for line in pp_train_data_reader_2:
    questions_number_array.append(line[0])
    questions_array.append(line[1])
    answer_number_array.append(line[2])
    answer_array.append(line[3])
#     label_array.append(line[4])  
label_array = binary_label
#print(questions_array)

In [10]:
concatqa_train = []
# print(len(questions_array))
for i in range (0, len(questions_array)):
    temp = questions_array[i] + " " + answer_array[i]
    concatqa_train.append(temp)
#print(concatqa_train)
# print(len(concatqa))

In [140]:
pandas_data = pandas.DataFrame(questions_array)
pandas_data.to_csv("questions_array.csv",sep=",",index=False,encoding="utf-8")
pandas_data = pandas.DataFrame(answer_array)
pandas_data.to_csv("answer_array.csv",sep=",",index=False,encoding="utf-8")

In [84]:
# pp_dev_data
pp_dev_data_file = open('pa_pp_test_data.csv', 'r', encoding = "utf8")
pp_dev_data_reader = csv.reader(pp_dev_data_file)

dev_binary_label = []
for line in pp_dev_data_reader:
    if line[4] == '2':
        dev_binary_label.append(1)
    else:
        dev_binary_label.append(0)
# print (dev_binary_label)

pp_dev_data_file_2 = open('pa_pp_test_data.csv', 'r', encoding = "utf8")
pp_dev_data_reader_2 = csv.reader(pp_dev_data_file_2)

dev_questions_number_array = []
dev_questions_array = []
dev_answer_number_array = []
dev_answer_array = []
dev_label_array = []

for line in pp_dev_data_reader_2:
    dev_questions_number_array.append(line[0])
    dev_questions_array.append(line[1])
    dev_answer_number_array.append(line[2])
    dev_answer_array.append(line[3])
#     dev_label_array.append(line[4])
#print(dev_questions_array)
dev_label_array = dev_binary_label

In [85]:
concatqa_dev = []
# print(len(dev_questions_array))
for i in range (0, len(dev_questions_array)):
    temp = dev_questions_array[i] + " " + dev_answer_array[i]
    concatqa_dev.append(temp)
# print(concatqa_dev)

# Step 2 & 3: Feature Extraction & MLP
<div style="direction:rtl  ; font-family: 'Nunito', sans-serif;" > 
    پس از انجام مراحل پیش‌پردازش داده‌ها را برای مرحله استخراج ویژگی آماده‌سازی می‌کنیم. چون در صورت تمرین دسته‌بندی باینری خواسته شده که در آن کلاس‌های مورد تقاضا True  به ازای پاسخ‌های Good و False به ازای پاسخ‌های Bad و PotentiallyUseful است، در این گام مقادیر label_array را برای ورودی 2 به 1 و برای ورودی‌های 1 و 0 را به 0 تبدیل می‌کنیم. 
اغلب روش‌های یادگیری ماشین بر روی داده‌های عددی قابل اجرا هستند و برای استفاده و اجرای آن‌ها روی داده‌های متنی نیاز به تبدیل متون به مجموعه اعداد است. پس هدف رویکردهای مختلف تبدیل متن به بردارهای عددی، استخراج و انتخاب مجموعه‌ای از ویژگی‌های مناسب از متون زبان طبیعی است. لذا انتخاب ویژگی مرحله‌ای بسیار مهم در فعالیت ما به شمار می‌رود، زیرا در این مرحله باید واژه‌های کلیدی انتخاب ‌شوند تا به عنوان بهترین نمایش‌دهنده برای سند متنی مورد استفاده قرار بگیرند. اگر تعداد واژه‌های کلیدی انتخاب شده کم باشد صحت و کارایی سیستم تحت تاثیر قرار می‌گیرد و کاهش می‌یابد و در مقابل اگر تعداد واژه‌های کلیدی انتخاب شده زیاد باشد باعث کاهش کارایی سیستم در بعد زمان خواهد شد و سرعت آموزش در فاز آموزش پایین می‌آید. 
روش SelectKBest، K ویژگی با بهترین معیار آماری را نگه داشته و از باقی ویژگی‌ها صرف نظر می‌کند که در اینجا مقدار K می‌تواند به وسیله ما مشخص شود. 
    
در این پروژه پس از استخراج 50000 ویژگی با روش‌های ذکر شده بالا، برای انتخاب ویژگی از روش‌ SelectKBest با توابع آماری chi2، f_classif، f_regression و mutual_info_regression برای انتخاب ویژگی‌های مجموعه‌های آموزش، توسعه و تست استفاده کرده و مدل شبکه پرسپترون چند لایه را با ویژگی‌های بدست آمده از این روش‌ها از مجموعه آموزش، آموزش داده و مدل آموزش داده شده را بر روی مجموعه dev_data با پارامترهای مختلف اجرا کرده و سعی کردیم تا بهترین پیکربندی برای هر مدل که معیار score را بیشینه می‌کند را با تحلیل و تغییر دادن پیکربندی‌ها بدست آوریم. در جدول زیر معیار score برای منتخب برخی از روش‌های گوناگون که تست شده قابل ملاحظه است.
    
    
</div>

In [88]:
max_f = 6000

In [89]:
#vectorize by count
#, ngram_range = (1, 3)
count_vectorizer = CountVectorizer(max_features = max_f)
X_count_train = count_vectorizer.fit_transform(concatqa_train).toarray()
y_count_train = label_array
D_count_train = numpy.array(X_count_train)
F_train = numpy.array(y_count_train)
# terms = count_vectorizer.get_feature_names_out()
# print("len doc", count_vectorizer)

In [90]:
#vectorize by count
count_vectorizer = CountVectorizer(max_features = max_f)
X_count_dev = count_vectorizer.fit_transform(concatqa_dev).toarray()
y_count_dev = dev_label_array
D_count_dev = numpy.array(X_count_dev)
F_dev = numpy.array(y_count_dev)

In [91]:
tfidf_vectorizer = TfidfVectorizer(max_features = max_f)
X_tfidf_train = tfidf_vectorizer.fit_transform(concatqa_train).toarray()
y_tfidf_train = label_array
D_tfidf_train = numpy.array(X_tfidf_train)

In [92]:
tfidf_vectorizer = TfidfVectorizer(max_features = max_f)
X_tfidf_dev = tfidf_vectorizer.fit_transform(concatqa_dev).toarray()
#S_X_tfidf_dev = SelectKBest(chi2, k = 20)
y_tfidf_dev = dev_label_array
D_tfidf_dev = numpy.array(X_tfidf_dev)

In [ ]:
def BM25_as_relevence_score(dev_questions_array, dev_answer_array):
    # calculate BM25 as "fake" relevence score, return as list
    tokenized_corpus = [doc.split(" ") for doc in sentences]
    bm25 = BM25Okapi(tokenized_corpus)
    query = caption
    tokenized_query = query.split(" ")
    cap_doc_scores = bm25.get_scores(tokenized_query)
    # print(cap_doc_scores)
    rel = np.zeros(len(cap_doc_scores))
return rel

In [93]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression, mutual_info_regression

In [94]:
input_D_train = np.append(D_count_train, D_tfidf_train , axis=1)
input_D_dev = np.append(D_count_dev, D_tfidf_dev , axis=1)

In [95]:
def SelectKBest_options(input_feature_selection, k_feature, max_i):
    aaa = SelectKBest(input_feature_selection, k = k_feature).fit_transform(input_D_train, F_train)
    bbb = SelectKBest(input_feature_selection, k = k_feature).fit_transform(input_D_dev, F_dev)
    clf = MLPClassifier(random_state=1, max_iter=max_i).fit(aaa, F_train)
    clf.predict_proba(bbb[:1])
    clf.predict(bbb[:, :])
    clf.score(bbb, F_dev)
    print("Score with",input_feature_selection, "k=",k_feature, "max_iter=",max_i,"is:",clf.score(bbb, F_dev))

In [96]:
SelectKBest_options(chi2, 50, 20)
SelectKBest_options(chi2, 100, 20)
SelectKBest_options(chi2, 200, 20)
SelectKBest_options(chi2, 300, 20)
SelectKBest_options(chi2, 400, 20)
SelectKBest_options(chi2, 500, 20)
SelectKBest_options(chi2, 600, 20)
SelectKBest_options(chi2, 50, 50)
SelectKBest_options(chi2, 100, 50)
SelectKBest_options(chi2, 200, 50)
SelectKBest_options(chi2, 300, 50)
SelectKBest_options(chi2, 400, 50)
SelectKBest_options(chi2, 500, 50)
SelectKBest_options(chi2, 600, 50)

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function chi2 at 0x00000281FD138790> k= 50 max_iter= 20 is: 1.0


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function chi2 at 0x00000281FD138790> k= 100 max_iter= 20 is: 1.0


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


...

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function chi2 at 0x00000281FD138790> k= 400 max_iter= 50 is: 0.9156996587030717


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function chi2 at 0x00000281FD138790> k= 500 max_iter= 50 is: 0.956655290102389
Score with <function chi2 at 0x00000281FD138790> k= 600 max_iter= 50 is: 0.8303754266211604


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [70]:
SelectKBest_options(f_classif, 50, 20)
SelectKBest_options(f_classif, 100, 20)
SelectKBest_options(f_classif, 200, 20)
SelectKBest_options(f_classif, 300, 20)
SelectKBest_options(f_classif, 400, 20)
SelectKBest_options(f_classif, 500, 20)
SelectKBest_options(f_classif, 600, 20)
SelectKBest_options(f_classif, 50, 50)
SelectKBest_options(f_classif, 100, 50)
SelectKBest_options(f_classif, 200, 50)
SelectKBest_options(f_classif, 300, 50)
SelectKBest_options(f_classif, 400, 50)
SelectKBest_options(f_classif, 500, 50)
SelectKBest_options(f_classif, 600, 50)

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_classif at 0x00000281FD138670> k= 50 max_iter= 20 is: 0.6930327868852459


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_classif at 0x00000281FD138670> k= 100 max_iter= 20 is: 0.6864754098360656


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


...

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_classif at 0x00000281FD138670> k= 400 max_iter= 50 is: 0.5672131147540984


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_classif at 0x00000281FD138670> k= 500 max_iter= 50 is: 0.5045081967213115
Score with <function f_classif at 0x00000281FD138670> k= 600 max_iter= 50 is: 0.5557377049180328


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [71]:
SelectKBest_options(f_regression, 50, 20)
SelectKBest_options(f_regression, 100, 20)
SelectKBest_options(f_regression, 200, 20)
SelectKBest_options(f_regression, 300, 20)
SelectKBest_options(f_regression, 400, 20)
SelectKBest_options(f_regression, 500, 20)
SelectKBest_options(f_regression, 600, 20)
SelectKBest_options(f_regression, 50, 50)
SelectKBest_options(f_regression, 100, 50)
SelectKBest_options(f_regression, 200, 50)
SelectKBest_options(f_regression, 300, 50)
SelectKBest_options(f_regression, 400, 50)
SelectKBest_options(f_regression, 500, 50)
SelectKBest_options(f_regression, 600, 50)

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_regression at 0x00000281FD138940> k= 50 max_iter= 20 is: 0.6930327868852459


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_regression at 0x00000281FD138940> k= 100 max_iter= 20 is: 0.6864754098360656


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


...

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_regression at 0x00000281FD138940> k= 400 max_iter= 50 is: 0.5672131147540984


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Score with <function f_regression at 0x00000281FD138940> k= 500 max_iter= 50 is: 0.5045081967213115
Score with <function f_regression at 0x00000281FD138940> k= 600 max_iter= 50 is: 0.5557377049180328


C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [97]:
input_D_train = np.append(D_count_train, D_tfidf_train , axis=1)
input_D_dev = np.append(D_count_dev, D_tfidf_dev , axis=1)
clf = MLPClassifier(random_state=1, max_iter=50).fit(input_D_train, F_train)
clf.predict_proba(input_D_dev[:1])
clf.predict(input_D_dev[:, :])
clf.score(input_D_dev, F_dev)

C:\Users\Arya\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7491467576791809

In [98]:
numbers = []
for i in range (0, int((len(dev_questions_array)/10))):
    numbers = numbers + [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# print(numbers)
# print(len(numbers))
max_clf_predict_proba = np.amax(clf_predict_proba, 1)
col_1 = dev_questions_number_array
col_2 = dev_answer_number_array
col_3 = numbers 
col_4 = max_clf_predict_proba
col_5 = clf_predict_class

df1 = pd.DataFrame()
df1.insert(0, "Question Number", col_1, True)
df1.insert(1, "Answer Number", col_2, True)
df1.insert(2, "Predict Proba", col_4, True)
df1.insert(3, "Answer class", col_5, True)

df2 = df1.groupby(["Question Number"]).apply(lambda x: x.sort_values(["Predict Proba"], ascending = False)).reset_index(drop=True)
df2.insert(2, "Predict Rank", col_3, True)

df2.to_csv("IR-Final-Prj-Step_3.csv",sep=" ",header=True,index=False,encoding="utf-8")
df2

,Question Number,Answer Number,Predict Rank,Predict Proba,Answer class
0,Q388_R14,Q388_R14_C9,1,0.999339,0
1,Q388_R14,Q388_R14_C5,2,0.997436,0
2,Q388_R14,Q388_R14_C7,3,0.996494,0
3,Q388_R14,Q388_R14_C1,4,0.994583,0
4,Q388_R14,Q388_R14_C6,5,0.988482,0
...,...,...,...,...,...
2925,Q475_R9,Q475_R9_C2,6,0.999692,0
2926,Q475_R9,Q475_R9_C3,7,0.998782,0
2927,Q475_R9,Q475_R9_C4,8,0.998625,0
2928,Q475_R9,Q475_R9_C7,9,0.996250,0


# Step 4: Glove
<div style="direction:rtl  ; font-family: 'Nunito', sans-serif;" > 
    
ایده اصلی پشت جاسازی کلمه GloVe این است که رابطه بین کلمات را به کمک آمار بدست آوریم. برخلاف ماتریس رخداد، ماتریس co-occurrence می‌گوید که چند بار یک جفت کلمه خاص با هم اتفاق می‌افتد. هر مقدار در ماتریس co-occurrence نشان دهنده یک جفت کلمه است که با هم اتفاق می‌افتند.
برای پیاده¬سازی این قسمت از کتابخانه zeugma.embeddings استفاده کردیم. ابتدا یک transformer را با استفاده از تابع EmbeddingTransformer که پارامتر 'glove' به آن داده شده است می¬سازیم و سپس داده¬ی ورودی را به آن می¬دهیم که با این بازنمایی ویژگی¬های آن استخراج شود. در ادامه نیز خروجی این نمایش را به همراه داده¬ی های برچسب آموزشی به شبکه MLP می¬دهیم. 

</div>

In [ ]:
from zeugma.embeddings import EmbeddingTransformer

In [ ]:
glove = EmbeddingTransformer('glove')
X_glove_train = glove.transform(concatqa_train)
y_train = label_array

In [ ]:
D_glove_train = numpy.array(X_glove_train)
F_train = numpy.array(y_train)
glove_dev = EmbeddingTransformer('glove')
X_glove_dev = glove_dev.transform(concatqa_dev)
y_glove_dev = dev_label_array

In [ ]:
D_glove_dev = numpy.array(X_glove_dev)
F_dev = numpy.array(y_glove_dev)
input_D_train = D_glove_train
input_D_dev = D_glove_dev

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=100).fit(input_D_train, F_train)
clf.predict_proba(input_D_dev[:1])
clf.predict(input_D_dev[:5, :])
clf.score(input_D_dev, F_dev)

In [99]:
clf_predict_proba = clf.predict_proba(input_D_dev[:])
clf_predict_class = clf.predict(input_D_dev[:, :])
numbers = []
for i in range (0, int((len(dev_questions_array)/10))):
    numbers = numbers + [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# print(numbers)
# print(len(numbers))
max_clf_predict_proba = np.amax(clf_predict_proba, 1)
col_1 = dev_questions_number_array
col_2 = dev_answer_number_array
col_3 = numbers 
col_4 = max_clf_predict_proba
col_5 = clf_predict_class

df3 = pd.DataFrame()
df3.insert(0, "Question Number", col_1, True)
df3.insert(1, "Answer Number", col_2, True)
df3.insert(2, "Predict Proba", col_4, True)
df3.insert(3, "Answer class", col_5, True)

df4 = df3.groupby(["Question Number"]).apply(lambda x: x.sort_values(["Predict Proba"], ascending = False)).reset_index(drop=True)
df4.insert(2, "Predict Rank", col_3, True)

df4.to_csv("IR-Final-Prj-Step_44.csv",sep=" ",header=True,index=False,encoding="utf-8")
df4

,Question Number,Answer Number,Predict Rank,Predict Proba,Answer class
0,Q388_R14,Q388_R14_C9,1,0.999339,0
1,Q388_R14,Q388_R14_C5,2,0.997436,0
2,Q388_R14,Q388_R14_C7,3,0.996494,0
3,Q388_R14,Q388_R14_C1,4,0.994583,0
4,Q388_R14,Q388_R14_C6,5,0.988482,0
...,...,...,...,...,...
2925,Q475_R9,Q475_R9_C2,6,0.999692,0
2926,Q475_R9,Q475_R9_C3,7,0.998782,0
2927,Q475_R9,Q475_R9_C4,8,0.998625,0
2928,Q475_R9,Q475_R9_C7,9,0.996250,0


# Step 5: BERT Finetuning


In [ ]:
from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import load_model

In [ ]:
# define model
# text/text pair classification
model = BertClassifier()   
# text/text pair regression
# model = BertRegressor()       
# token sequence classification
# model = BertTokenClassifier() 

In [ ]:
# finetune model
model.fit(X_train, y_train)
# make predictions
y_pred = model.predict(X_test)
# make probabilty predictions
y_pred = model.predict_proba(X_test)
# score model on test data
model.score(X_test, y_test)

In [100]:
# save model
df6 = df3.groupby(["Question Number"]).apply(lambda x: x.sort_values(["Predict Proba"], ascending = False)).reset_index(drop=True)
df6.insert(2, "Predict Rank", col_3, True)
df6.to_csv("IR-Final-Prj-Step_44.csv",sep=" ",header=True,index=False,encoding="utf-8")
df6
df6.to_csv("IR-Final-Prj-Step_55.csv",sep=" ",header=True,index=False,encoding="utf-8")
model.save(savefile)

,Question Number,Answer Number,Predict Rank,Predict Proba,Answer class
0,Q388_R14,Q388_R14_C9,1,0.999339,0
1,Q388_R14,Q388_R14_C5,2,0.997436,0
2,Q388_R14,Q388_R14_C7,3,0.996494,0
3,Q388_R14,Q388_R14_C1,4,0.994583,0
4,Q388_R14,Q388_R14_C6,5,0.988482,0
...,...,...,...,...,...
2925,Q475_R9,Q475_R9_C2,6,0.999692,0
2926,Q475_R9,Q475_R9_C3,7,0.998782,0
2927,Q475_R9,Q475_R9_C4,8,0.998625,0
2928,Q475_R9,Q475_R9_C7,9,0.996250,0


In [68]:
# ********************************************************************
# # datafile = open('train_data.csv', 'r', encoding="utf8")
# myreader = csv.reader(datafile)
# X = []
# y = []
# temp = ["0", "0"]
# for qqaa in myreader:
#     tx = [" ", " "]
#     ty = 5
#     temp[0] = qqaa[1]
# #     print(temp[0])
#     temp[1] = qqaa[3]
# #     print(temp[1])
#     tx = [temp[0], temp[1]]
#     ty = qqaa[4]
#     X.append(tx)
#     y.append(ty)
# print(X[20], X[21])
# print(X[2])
# print(y[2])
# selector = SelectFromModel(estimator=LogisticRegression()).fit(XX, yy)
# selector.estimator_.coef_
# print(get_KB_selector("f_classif", d, f, k))
# //
# selector = get_KB_selector("f_classif", d, f, k)
# d = selector.transform(d)
# XX = numpy.array(d)
# yy = numpy.array(f)
# clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
# clf.fit(XX, yy)
# td, tf = reader.readVTestData('./dm_final/data/V1_ECFP4.csv')
# td = selector.transform(td)
# r = clf.predict(td)
# y_true = tf
# y_pred = r.tolist()
# utils.show_result(y_true, y_pred)
# with open('X.txt', 'w') as f:
#     f.write(str(X))

In [66]:
# fe_temp = ["0", "0"]
# for qqaa in myreader:
#     tx = [" ", " "]
#     ty = 5
#     fe_temp[0] = qqaa[1]
# #     print(temp[0])
#     fe_temp[1] = qqaa[3]
# #     print(temp[1])
#     tx = [fe_temp[0], fe_temp[1]]
#     ty = qqaa[4]
#     X.append(tx)
#     y.append(ty)
# print(X[20], X[21])
# print(X[2])
# print(y[2])